In [ ]:
import numpy as np # library to handle data in a vectorized manner
import pandas as pd # library for data analsysis
import json # library to handle JSON files

from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors
import pandas as pd
import matplotlib.pyplot as plt

print('Imported.')

In [ ]:
pip install lxml html5lib beautifulsoup4

In [ ]:
# Define Foursquare Credentials and Version
CLIENT_ID = 'V1JP0OQNXOSW0TMHAKMEP33U1UAWHCO2I3LZQLTI1MEA1D2Y' # your Foursquare ID
CLIENT_SECRET = 'Y2YU3WT2HRVMRR5H2RPXM3BOW5XMYDN3EIAO2A0MMHW0SEDN' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

In [ ]:
#Download Bangkok, Thailand list of Districts 
url = 'https://en.wikipedia.org/wiki/List_of_districts_of_Bangkok'
dfs = pd.read_html(url)
print(dfs[0])

In [ ]:
#Transform to Dataframe
df = dfs[0]

In [ ]:
#Fix spelling of the columns
df = df.rename(columns={"Popu-lation": "Population"})
df = df.rename(columns={"District(Khet)": "District"})
df = df.rename(columns={"Post-code": "Postcode"})

In [ ]:
#Clean table 
BKK_districts = df[['District','Postcode','Population','Latitude', 'Longitude']]

In [ ]:
#check if everything is ok
BKK_districts.head()

In [ ]:
#create an istrogram 
clr = "blue"
BKK_districts.groupby('District')['Population'].sum().plot.bar(figsize=(10,5), color=clr)
plt.title('Population  per District', fontsize = 20)
plt.xlabel('District', fontsize = 15)
plt.ylabel('Population',fontsize = 15)
plt.legend()
plt.show()

In [ ]:
#extract the first district to be use in a test 
df.loc[0, 'District']

In [ ]:
#test if you can extract Lat and Long
neighborhood_latitude = df.loc[0, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = df.loc[0, 'Longitude'] # neighborhood longitude value

neighborhood_name = df.loc[0, 'District'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

In [ ]:
#extract the data from Foursquare 
LIMIT = 500

radius = 1000

url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)
url # display URL

In [ ]:
#define the main functions used later

def geo_location(address):
    # get geo location of address
    geolocator = Nominatim(user_agent="foursquare_agent")
    location = geolocator.geocode(address)
    latitude = location.latitude
    longitude = location.longitude
    return latitude,longitude


def get_venues(lat,lng):
    #set variables
    radius=1000
    LIMIT=500
    #url to fetch data from foursquare api
    url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
    # get all the data
    results = requests.get(url).json()
    venue_data=results['response']['groups'][0]['items']
    venue_details=[]
    for row in venue_data:
        try:
            venue_id=row['venue']['id']
            venue_name=row['venue']['name']
            venue_category=row['venue']['categories'][0]['name']
            venue_details.append([venue_id,venue_name,venue_category])
        except KeyError:
            pass
    column_names=['ID','Name','Category']
    df = pd.DataFrame(venue_details,columns=column_names)
    return df


def get_venue_details(venue_id):
    #url to fetch data from foursquare api
    url = 'https://api.foursquare.com/v2/venues/{}?&client_id={}&client_secret={}&v={}'.format(
            venue_id,
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION)
    # get all the data
    results = requests.get(url).json()
    print(results)
    venue_data=results['response']['venue']
    venue_details=[]
    try:
        venue_id=venue_data['id']
        venue_name=venue_data['name']
        venue_likes=venue_data['likes']['count']
        venue_rating=venue_data['rating']
        venue_tips=venue_data['tips']['count']
        venue_details.append([venue_id,venue_name,venue_likes,venue_rating,venue_tips])
    except KeyError:
        pass
    column_names=['ID','Name','Likes','Rating','Tips']
    df = pd.DataFrame(venue_details,columns=column_names)
    return df

In [ ]:
# prepare neighborhood list that contains Italian restaurants

column_names=['District', 'ID','Name']
ita_rest_bkk=pd.DataFrame(columns=column_names)
count=1
for row in BKK_districts.values.tolist():
    District, Postcode, Population, Latitude, Longitude=row
    venues = get_venues(Latitude,Longitude)
    italian_restaurants=venues[venues['Category']=='Italian Restaurant']   
    print('(',count,'/',len(BKK_districts),')','Italian Restaurant in '+District+':'+str(len(italian_restaurants)))
    print(row)
    for resturant_detail in italian_restaurants.values.tolist():
        id, name , category=resturant_detail
        ita_rest_bkk = ita_rest_bkk.append({'District': District,
                                                 'ID': id,
                                                'Name' : name
                                               }, ignore_index=True)
    count+=1


In [ ]:
#make a back-up in .csv file 
ita_rest_bkk.to_csv('ita_rest_bkk_tocsv1.csv')

In [ ]:
#check data
ita_bkk = pd.read_csv('ita_rest_bkk_tocsv1.csv')
ita_rest_bkk.tail()

In [ ]:
#check the data size
ita_rest_bkk.shape

In [ ]:
#Let's Visualize the distribution of the Italian Restaurants
ita_rest_bkk.groupby('District')['ID'].count().plot.bar(figsize=(10,5), color=clr)
plt.title('Italian Restaurants per District', fontsize = 20)
plt.xlabel('District', fontsize = 15)
plt.ylabel('Italian Restaurants',fontsize = 15)
plt.legend()
plt.show()

In [ ]:
column_names=['District', 'ID','Name','Likes','Rating','Tips']
ita_rest_stats_bkk=pd.DataFrame(columns=column_names)
count=1
for row in ita_rest_bkk.values.tolist():
    Disrict, ID, Name=row
    try:
        venue_details=get_venue_details(ID)
        print(venue_details)
        id,name,likes,rating,tips=venue_details.values.tolist()[0]
    except IndexError:
        print('No data available for id=',ID)
        # we will assign 0 value for these resturants as they may have been 
        #recently opened or details does not exist in FourSquare Database
        id,name,likes,rating,tips=[0]*5
    print('(',count,'/',len(ita_rest_bkk),')','processed')
    ita_rest_stats_bkk = ita_rest_stats_bkk.append({'Disrict': Disrict,
                                                'ID': id,
                                                'Name' : name,
                                                'Likes' : likes,
                                                'Rating' : rating,
                                                'Tips' : tips
                                               }, ignore_index=True)
    count+=1
ita_rest_stats_bkk.head()

In [ ]:
#make a back-up in .csv file 
ita_rest_stats_bkk.to_csv('ita_rest_stats_bkk_csv.csv')

In [ ]:
#check the data format
ita_rest_stats_bkk.shape

In [ ]:
#trasform data format
ita_rest_stats_bkk['Likes'] = ita_rest_stats_bkk['Likes'].astype('float64')
ita_rest_stats_bkk['Tips'] = ita_rest_stats_bkk['Tips'].astype('float64')
ita_rest_stats_bkk.info()

In [ ]:
# Resturant with maximum Likes
ita_rest_stats_bkk.iloc[ita_rest_stats_bkk['Likes'].idxmax()]

In [ ]:
# Resturant with maximum Ratings
ita_rest_stats_bkk.iloc[ita_rest_stats_bkk['Rating'].idxmax()]

In [ ]:
#create a table groupping the district 
BKK_districts_stats=ita_rest_stats_bkk.groupby('Disrict',as_index=False).mean()[['Disrict','Rating']]
BKK_districts_stats.columns=['District','Average Rating']
BKK_districts_stats.sort_values(['Average Rating'],ascending=False).head(50)

In [ ]:
#create a table groupping the district 
BKK_districts_stats_L=ita_rest_stats_bkk.groupby('Disrict',as_index=False).mean()[['Disrict','Likes']]
BKK_districts_stats_L.columns=['District','Average Likes']
BKK_districts_stats_L.sort_values(['Average Likes'],ascending=False).head(50)

In [ ]:
#Let's Visualize the distribution of the Rating
BKK_districts_stats.groupby('District')['Average Rating'].mean().plot.bar(figsize=(10,5), color=clr)
plt.title('Average Rating per District', fontsize = 20)
plt.xlabel('District', fontsize = 15)
plt.ylabel('Average Rating',fontsize = 15)
plt.legend()
plt.show()


In [ ]:
#Let's Visualize the distribution of the Likes
BKK_districts_stats_L.groupby('District')['Average Likes'].mean().plot.bar(figsize=(10,5), color=clr)
plt.title('Average Likes per District', fontsize = 20)
plt.xlabel('District', fontsize = 15)
plt.ylabel('Average Likes',fontsize = 15)
plt.legend()
plt.show()
